In [3]:
# 외부 수집데이터(크롤링, API, CSV)들과 부동산 시가총액 단위별 상관관계(전체 시계열 - 주간단위) 산출

import pandas as pd
import numpy as np
import os
import glob

from datetime import date
import os

tdy = str(date.today().isoformat()).replace('-','')

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
aptaprice = pd.read_csv("D:\\시세모니터링\\융합분석\\과거 시가총액 조회\\dat_all_202068.csv", index_col=0)
aptaprice = aptaprice.rename(columns={"조사일" : 'date'})
# aptaprice.index = aptaprice["date"]
# aptaprice = aptaprice.drop(columns=["date"])
aptaprice["date"] = aptaprice["date"].astype(str).str[:4] + "-" + aptaprice["date"].astype(str).str[4:6] + "-" + aptaprice["date"].astype(str).str[-2:]
sgg_lst = pd.unique(aptaprice.지역명)
aptaprice

,date,지역명,aptprice_all_sum
0,1997-01-15,전국,1.285452e+10
1,1997-02-15,전국,1.301010e+10
2,1997-03-15,전국,1.313440e+10
3,1997-04-15,전국,1.322408e+10
4,1997-05-15,전국,1.337931e+10
...,...,...,...
33278,2020-06-05,파주시,2.043878e+09
33279,2020-06-05,평택시,2.125171e+09
33280,2020-06-05,포천시,1.834887e+08
33281,2020-06-05,하남시,2.822876e+09


In [8]:
from tqdm import tqdm_notebook
path = "D:/시세모니터링/융합분석/DATA/20200611/"
file = glob.glob(path+ "\\*.csv")

## 결과저장용 dataframe 생성 (index = 시군구, 컬럼수 : 크롤링 데이터 수 대로 0 을 채워넣음)
empt_lst_idx = pd.DataFrame(np.zeros(74 * len(file)).reshape(74,-1), columns = [i.split('.csv')[0].split('\\')[1] for i in file], index = sgg_lst)


for i in tqdm_notebook(file):
    list_temp = pd.read_csv(i)
    list_temp = list_temp.iloc[:,:2]
    list_temp = list_temp.rename(columns={"Date" : 'date'})
    
    try :
        list_temp.iloc[:,1] = list_temp.iloc[:,1].str.replace(",","").astype(float)
    except :
        pass
    
    apt_merge = pd.merge(aptaprice, list_temp, how = 'inner')
#     apt_merge.to_csv(path+"apt_gu_" + i.split("\\")[1].split(".csv")[0]+ '.csv')
    
    for num,sgg_nm in enumerate(sgg_lst):
        corr_gu = apt_merge[apt_merge["지역명"] == sgg_nm].sort_values("date")
        res = corr_gu.iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=True)[0]
        
        #결과저장용 df에 insert수행
        empt_lst_idx.loc[empt_lst_idx.index == sgg_nm, corr_gu.columns[3]] = res
        
        if ((res > 0.7) | (res < -0.7)) & (len(corr_gu) > 1000):
            # 상관관계 0.5 이상, 건수 500건 이상
#             apt_merge.to_csv(path+"apt_gu_" + i.split("\\")[1].split(".csv")[0]+ '.csv')
            print(str(num) + "/ 25")
            print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0] + " 의 상관관계")
            print("# 대상 Row수 : " + str(len(corr_gu)) + "건")
            print("# 관계 : " + str(res))
            print("\n")
                        
#         elif ((res < 0.5) | (res > -0.5)) :
#             print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0])
#             print("# 상관관계 낮음 : " + str(res))
#             print("# 제외 대상 Row수 : " + str(len(corr_gu)) + "건")
#             print("\n")
            
        elif len(corr_gu) < 500:
            print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0])
            print("# 제외 대상 Row수 : " + str(len(corr_gu)) + "건")
            print("\n")

# 결과저장용 df CSV 저장
empt_lst_idx.T.to_csv("sgg_gu_cor_res_"+ tdy + ".csv", encoding="CP949")

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


0/ 25
# 전국 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8329110509821318


1/ 25
# 강원도 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.851424698356424


2/ 25
# 경기도 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8143732877924984


3/ 25
# 경상남도 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8679855853958615


4/ 25
# 경상북도 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8764706828322267


5/ 25
# 광주광역시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8611228833633569


6/ 25
# 대구광역시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.858157055516663


7/ 25
# 대전광역시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8571749795794077


8/ 25
# 부산광역시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8583940474646354


9/ 25
# 서울특별시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7818459947214272


10/ 25
# 인천광역시 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8475853433625589


11/ 25
# 전라북도 시가총액과 CD(91일) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8523519377666572


12/ 25
# 충청남도 시가총액과 CD(91일

50/ 25
# 부천시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8851224275339512


51/ 25
# 성남시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8429753613973867


52/ 25
# 수원시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.9027011315421907


53/ 25
# 시흥시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8600466258048506


54/ 25
# 안산시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8761322418601282


55/ 25
# 안양시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8505454855958919


56/ 25
# 용인시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8517223459850277


57/ 25
# 의왕시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.9125441755546224


58/ 25
# 의정부시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8181424578141816


59/ 25
# 평택시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8890919077518268


60/ 25
# 하남시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.7333087531882043


61/ 25
# 광주시 시가총액과 CMA(수시형) 의 상관관계
# 대상 Row수 : 1008건
# 관계 : -0.8487173110343038


62/ 25
# 파주시 시가

0/ 25
# 전국 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8706425327098838


1/ 25
# 강원도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8841145269831342


2/ 25
# 경기도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8821138488517455


3/ 25
# 경상남도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8797217925334133


4/ 25
# 경상북도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1052건
# 관계 : 0.8546411390535842


5/ 25
# 광주광역시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8474473501164396


6/ 25
# 대구광역시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8001513068721048


7/ 25
# 대전광역시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8867226740064958


8/ 25
# 부산광역시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8446077657715249


9/ 25
# 서울특별시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8376585407045201


10/ 25
# 인천광역시 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8973373470150258


11/ 25
# 전라북도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.9261503050156434


12/ 25
# 충청남도 시가총액과 gold 의 상관관계
# 대상 Row수 : 1060건
# 관계 : 0.8806661828152

49/ 25
# 남양주시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9391722777181731


50/ 25
# 부천시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9209994763546622


51/ 25
# 성남시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.8866322918938631


52/ 25
# 수원시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9071675620839067


53/ 25
# 시흥시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.8988527017415611


54/ 25
# 안산시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9232703358680443


55/ 25
# 안양시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9017317341319916


56/ 25
# 용인시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9199259269295198


57/ 25
# 의왕시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9308381821344482


58/ 25
# 의정부시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.8921746338556547


59/ 25
# 평택시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.9013502954533408


60/ 25
# 하남시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1062건
# 관계 : 0.7176898438641923


61/ 25
# 광주시 시가총액과 kospi 의 상관관계
# 대상 Row수 : 1046건
# 관계 : 0.915

...

# 고양시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 과천시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 광명시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 구리시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 군포시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 김포시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 남양주시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 부천시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 성남시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 수원시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 시흥시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 안산시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 안양시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 용인시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 의왕시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 의정부시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 평택시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 하남시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 광주시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 파주시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 이천시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 동두천시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 안성시 시가총액과 국고채(30년)
# 제외 대상 Row수 : 375건


# 오산시 시가

26/ 25
# 노원구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7780604543148981


27/ 25
# 도봉구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7742598654959143


28/ 25
# 동대문구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8410315832885855


29/ 25
# 동작구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8257325416581488


30/ 25
# 마포구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.823741912412066


31/ 25
# 서대문구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8028856884595124


32/ 25
# 서초구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8106431331239491


33/ 25
# 성동구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7931239892591666


34/ 25
# 성북구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8197675481998284


35/ 25
# 송파구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7754406106416147


36/ 25
# 양천구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7530551282485044


37/ 25
# 영등포구 시가총액과 국고채(5년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.781218540918759


38/ 25
# 용산구 시가총액과 국고채(5년) 

0/ 25
# 전국 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8225488588585635


1/ 25
# 강원도 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.852233055162474


2/ 25
# 경기도 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7954634743289781


3/ 25
# 경상남도 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8588451353921351


4/ 25
# 경상북도 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8669916456103728


5/ 25
# 광주광역시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8604438856322277


6/ 25
# 대구광역시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8587933445002879


7/ 25
# 대전광역시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8335861053090746


8/ 25
# 부산광역시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8566626906843384


9/ 25
# 서울특별시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7721395305965081


10/ 25
# 인천광역시 시가총액과 콜금리(1일, 중개회사거래) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8460801426259015


11/ 25
# 전라북도 시가총액과

54/ 25
# 안산시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8258145454930235


55/ 25
# 안양시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7780584569501828


56/ 25
# 용인시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7751405594315917


57/ 25
# 의왕시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8730522518739235


58/ 25
# 의정부시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7714234824971813


59/ 25
# 평택시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8609878364252282


60/ 25
# 하남시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7074009063418061


61/ 25
# 광주시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7830926892240498


62/ 25
# 파주시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8200225661294689


63/ 25
# 이천시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8699204722636007


64/ 25
# 동두천시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8192117846860001


65/ 25
# 안성시 시가총액과 통안증권(1년) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.8693616141918328


66/ 25
# 오산시 시

# 관계 : -0.7025231842058667


28/ 25
# 동대문구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7924941383942086


29/ 25
# 동작구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7781639661620443


30/ 25
# 마포구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7790637017671206


31/ 25
# 서대문구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7576256343147


32/ 25
# 서초구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7708787452526668


33/ 25
# 성동구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.75073843813656


34/ 25
# 성북구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7665031070134404


35/ 25
# 송파구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7183094897433194


36/ 25
# 양천구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7029106426847995


37/ 25
# 영등포구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7264573985634587


38/ 25
# 용산구 시가총액과 회사채(3년,AA-) 의 상관관계
# 대상 Row수 : 1007건
# 관계 : -0.7284919813079629


39/ 25
# 은평구 시가총액과 회사채(3년,AA-) 의 상관관계


In [6]:
res_list = ["nq-100",
            "south-korea-1-year-bond-yield",
            "south-korea-2-year-bond-yield",
            "south-korea-3-year-bond-yield",
            "south-korea-5-year-bond-yield",
            "u.s.-10-year-bond-yield",
            "u.s.-30-year-bond-yield",
            "kr_bs_rate",
            "us_bs_rate",
           "kospi",
            "kosdaq",
           "gold",
           "smallcap-2000",
            "KOSDAQ_거래대금",
            "KOSPI_거래대금시간외",
            "KOSPI_시가총액",
            'shanghai-composite',
            'volatility-s-p-500',
            'us-dollar-index',
            'hang-sen-40',
            "MMF(7일)",
            "CMA(수시형)",
            "통안증권(2년)",
            "통안증권(1년)",
            "CD(91일)",
            "콜금리(1일, 전체거래)",
            "회사채(3년,AA-)",
            "us-30",
            "us-spx-500"
           ]

In [7]:
reslt = aptaprice

for a in file:
    if (a.split("\\")[1].split(".csv")[0] in res_list) == True:
        temp_res = pd.read_csv(a)
        temp_res = temp_res.iloc[:,:2]
        temp_res = temp_res.rename(columns={"Date" : 'date'})
        
        reslt = pd.merge(reslt, temp_res, how = 'inner')
reslt.to_csv("merge_ext_" + tdy + ".csv", index=False)
reslt

,date,지역명,aptprice_all_sum,CD(91일),CMA(수시형),gold,hang-sen-40,kosdaq,KOSDAQ_거래대금,kospi,...,u.s.-30-year-bond-yield,us-30,us-dollar-index,us-spx-500,us_bs_rate,volatility-s-p-500,"콜금리(1일, 전체거래)",통안증권(1년),통안증권(2년),"회사채(3년,AA-)"
0,2007-01-05,전국,1.367448e+11,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
1,2007-01-05,강원도,1.070694e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
2,2007-01-05,경기도,4.684591e+10,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
3,2007-01-05,경상남도,3.523327e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
4,2007-01-05,경상북도,1.710930e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40177,2020-06-05,파주시,2.043878e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40178,2020-06-05,평택시,2.125171e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40179,2020-06-05,포천시,1.834887e+08,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40180,2020-06-05,하남시,2.822876e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
